# ArcGIS Online WAB Widget Audit
Description: This will audit all of the Web App Builder Aplications in an organization and the widgets within them.
 
Beginning with: AGOWABWidgetAuditPythonAPI.py
 
Created on: 10/18/2023
 
Purpose: This will audit all of the web App Builder Applications in an organization,
    and provide a list of the widgets within them. You can then cross reference the list 
    of widgets with the following blog, in order to prioritize the order with which 
    you migrate your Web App Builder Applications to Experience Builder.
    https://community.esri.com/t5/arcgis-experience-builder-documents/functionality-matrix-for-web-appbuilder-and/ta-p/1113766
 
Authored By: Joe Guzi 

Inspired By: Brandon Wise
 
Previous Production Date: 10/18/23     
Production Date: 10/24/23

# Import Libraries

In [ ]:
# import
from arcgis.gis import GIS
from arcgis.gis.server import Server
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Connect to ArcGIS Online

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass('Password: ')

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

# Load Functions

In [ ]:
# Functions
def ReturnWidgetData(Widget):
    global ItemID
    global ItemTitle
    global ItemOwner
    global ItemTags
    global ItemContentStatus
    global WABWA
    WidgetName = Widget.get("name")
    Visibility = Widget.get("visible")

    if WidgetName != None and Visibility != False:
        print(str(WidgetName) + "\n")
        WABWA += ItemID + ","
        WABWA += ItemTitle + ","
        WABWA += ItemOwner + ","
        WABWA += ItemTags + ","
        WABWA += ItemContentStatus + ","
        WABWA += WidgetName + ","
        WABWA += "\n"
        
def LoopThroughWidgetList(WidgetDataVariable):
    global ItemID
    global ItemTitle
    global ItemOwner
    global ItemTags
    global ItemContentStatus
    global WABWA
    WidgetList = WidgetDataVariable.get("widgets")
    if WidgetList != None:
        for Widget in WidgetList:
            #print(Widget)
            ReturnWidgetData(Widget)
    GroupList = WidgetData.get("groups")
    if GroupList != None:
        for Group in GroupList:
            WidgetList = Group.get("widgets")
            for Widget in WidgetList:
                #print(Widget)
                if WidgetList != None:
                    ReturnWidgetData(Widget)
            
print("End Load Functions")

# Audit Section

In [ ]:
print("Find WAB Apps")
WebAppAudit = gis.content.search(query="", item_type="Web Mapping Application", max_items = 10000)
WABWA = "" # Web App Builder Widget Audit text

print("Begining to Loop Through Items" + "\n")
for Item in WebAppAudit:
    print("Entering Item Loop" + "\n")
    print(str(Item) + "\n")
    ItemID = str(Item.get("id"))
    TitelPrep = str(Item.get("title"))
    ItemTitle = TitelPrep.replace(',', "")
    ItemOwner = str(Item.get("owner"))
    TagPrep = str(Item.get("tags"))
    ItemTags = TagPrep.replace(',', "|")
    ItemContentStatus = str(Item.get("contentStatus"))
    
    JSON = Item.get_data(try_json=False) # when it breaks it is breaking here
    if JSON != None:
        ItemData = json.loads(JSON)
        #print("Item Data: " + str(ItemData)+ "\n")
        for ItemInfo in ItemData:
            # Operational Layers
            if ItemInfo == "widgetOnScreen":
                print("Looping Through Widgets On Screen" + "\n")
                WidgetData = ItemData.get("widgetOnScreen")
                LoopThroughWidgetList(WidgetData)
            if ItemInfo == "widgetPool":
                print("Looping Through Widget Pool" + "\n")
                WidgetData = ItemData.get("widgetPool")
                LoopThroughWidgetList(WidgetData)
        
print("Audit Section Complete!")

# Write the Web App Builder Widget Audit Results to Excel

In [ ]:
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("AppID,AppTitle,Owner,Tags,Status,Widget\n")
    output.writelines(WABWA)
    print("Wrtie data To CSV File Complete!")